In [60]:
from pathlib import Path
from typing import List
import pandas as pd
from pandas import DataFrame

import seaborn as sns

In [61]:
SCHEMA: int = 1
TABLE_NAME: int = 2
COLUMN_NAME: int = 4
DATA_TYPE: int = 7

data_path: Path = Path("../data/")

### Processing

In [62]:
SAP_data_path: Path = data_path / "data.csv"
SAP_data: DataFrame = pd.read_csv(SAP_data_path, header=None, sep=",", error_bad_lines=False)
filtered_by_scheme: DataFrame = SAP_data[SAP_data[SCHEMA] == "SAPISP"]
grouped_by_column: DataFrame = filtered_by_scheme\
    .groupby([DATA_TYPE, COLUMN_NAME, TABLE_NAME], as_index=False)[8].count()

b'Skipping line 2937727: expected 37 fields, saw 38\n'
b'Skipping line 3073097: expected 37 fields, saw 39\n'
b'Skipping line 3084376: expected 37 fields, saw 39\nSkipping line 3087083: expected 37 fields, saw 39\n'
b'Skipping line 3100565: expected 37 fields, saw 39\nSkipping line 3103761: expected 37 fields, saw 39\nSkipping line 3103764: expected 37 fields, saw 39\n'
/Users/linusheinzl/Documents/Semester2/DatabaseSystems/column-compression/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (12,14,25,30,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [63]:
isp_plan_analyses: DataFrame = pd.read_csv(data_path / 'isp.csv')
plan_executions = pd.read_csv(data_path / 'plan_execution_counts.csv')
column_accesses = pd.merge(isp_plan_analyses, plan_executions)

In [64]:
tables: List[str] = column_accesses["OBJECT_NAME"].unique()
only_valid_tables: DataFrame = grouped_by_column[grouped_by_column[TABLE_NAME].isin(tables)]

In [65]:
with_type_info: DataFrame = pd.merge(column_accesses, only_valid_tables,
                  left_on=["OBJECT_NAME", "COLUMN_NAME"], right_on=[TABLE_NAME, COLUMN_NAME])

### Results

In [66]:
with_type_info.groupby(["USAGE", DATA_TYPE])["EXECUTION_COUNT"].sum()

USAGE        7        
AGGREGATION  DECIMAL        2112038
             NVARCHAR        270959
             SMALLINT            76
FILTER       DECIMAL         109823
             NVARCHAR      40094250
GROUPING     DECIMAL        3481070
             NVARCHAR      45310567
             VARBINARY       272736
JOIN         NVARCHAR       8628297
PROJECT      DECIMAL       15871133
             NVARCHAR     109045298
             SMALLINT            15
             VARBINARY       290093
SORTING      NVARCHAR         10797
Name: EXECUTION_COUNT, dtype: int64

In [67]:
with_type_info.groupby([DATA_TYPE])["EXECUTION_COUNT"].sum()

7
DECIMAL       21574064
NVARCHAR     203360168
SMALLINT            91
VARBINARY       562829
Name: EXECUTION_COUNT, dtype: int64

In [68]:
only_valid_tables.groupby([DATA_TYPE])[8].count()

7
DECIMAL       448
DOUBLE          5
INTEGER         3
NVARCHAR     2232
SMALLINT        9
VARBINARY       4
Name: 8, dtype: int64